In [2]:
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
import errno
from IPython.display import display, HTML
import subprocess
#import pdfkit as pdf

#HTML(df.to_html())
#display(df)

hpi_red = '#b00639'
hpi_blue = '#007a9e'

def create_folder(filename):
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

def data_frame_to_pdf(df):
    filename = 'out.tex'
    pdffile = 'out.pdf'
    outname = 'out.png'

    template = r'''\documentclass[preview]{{standalone}}
    \usepackage{{booktabs}}
    \begin{{document}}
    {}
    \end{{document}}
    '''

    create_folder(filename)
    with open(filename, 'w') as f:
        f.write(template.format(df.to_latex()))

    subprocess.call(['pdflatex', filename])
    subprocess.call(['convert', '-density', '300', pdffile, '-quality', '90', outname])
    
def data_frame_to_tex(df, filename):
    create_folder(filename)
    with open(filename, 'w') as f:
        f.write(df.to_latex())

def get_column_names(benchmark): 
    file = 'results/' + benchmark + '_results.csv.json'
    table_info = json.load(open(file,'r'))
    #print(json.dumps(table_info, indent=2, sort_keys=False))
    column_names = []
    for column in table_info["columns"]:
        column_names.append(column["name"])
    return column_names

def get_results(benchmark):
    file = 'results/' + benchmark + '_results.csv'
    return pd.read_csv(file, header=None, names=get_column_names(benchmark))

def percent(x, pos):
    'The two args are the value and tick position'
    return ('%i' % (x * 100)) + "%"

# Columns: sample_size, row_count, distinct_values, data, estimation_technique, error, occurrences 
def select(data, row_count, distinct_values, data_name, estimation_tec):
    return data[(data.row_count == row_count) &
                (data.distinct_values == distinct_values) &
                (data.data_name == data_name) & 
                (data.estimation_technique == estimation_tec)]

def get_mean_squared_error(errors, probabilities):
    mse = 0.0
    for i in range(0, len(errors)):
        error = errors.iloc[i]
        probability = probabilities.iloc[i]
        mse += probability * error * error
        
    return mse

def get_mean_error(errors, probabilities):
    me = 0.0
    for i in range(0, len(errors)):
        error = errors.iloc[i]
        probability = probabilities.iloc[i]
        me += probability * abs(error)
        #print("error: " + str(error) + ", probability: " + str(probability))
        
    return me

def misestimation_evaluation(row_count, distinct_values, data_name, estimation_tec):
    #percent_formatter = FuncFormatter(percent)
    #ax.yaxis.set_major_formatter(percent_formatter)
    
    #Prepare the data
    data = get_results("misestimation")
    selection = select(data, row_count, distinct_values, data_name, estimation_tec)
    if len(selection) == 0:
        return
    sample_size = selection["sample_size"].iloc[0]
    errors = selection["error"]
    probabilities = selection["occurrences"].apply(lambda x: x / sample_size)
    biggest_error = max(max(errors), 0)
    lowest_error = min(min(errors), 0)
    x_data = range(lowest_error, biggest_error + 1)
    y_data = np.zeros(biggest_error - lowest_error + 1)
    for i in range(0, len(errors)):
        y_data[errors.iloc[i] - lowest_error] = probabilities.iloc[i]     
    #one_off_probability = 0
    #if(len(y_data) > 1):
    #    one_off_probability = y_data[1]
    #correct_probability = y_data[0]
    mean_error = get_mean_error(errors, probabilities)
    mean_square_error = get_mean_squared_error(errors, probabilities)
    #filter_size_bits = pow(2, quotient_size) * (2 + remainder_size)
    #filter_size_bytes = filter_size_bits / 8
    #bits_per_value = filter_size_bits / row_count
    column_size = row_count * 2
    
    # Print some Information
    print("Estimation Technique: " + estimation_tec)
    print("Sample Size: " + str(sample_size))
    #print("One off error probability: " + str(one_off_probability))
    print("Mean Error: " + str(mean_error))
    print("Mean Squared Error: " + str(mean_square_error))
    #print("Correct " + str(correct_probability * 100) + "%" + " of the time")
    #print("Filter Size [kB]: " + str(filter_size_bytes / 1000))
    #print("Bits per Value: " + str(bits_per_value))
    print("Column Size [kB](uint16_t): "  + str(column_size / 1000))
    print("")
    
    # Configure the plot
    #title = "Overcount probabilites"
    #subtitle = "Value Count: " + str(row_count) \
    #            + ", Distinct Values: " + str(distinct_values) + ", Data: " + data_name + '\n'\
    #            + "Quotient Size: " + str(quotient_size) + ", Remainder Size: " + str(remainder_size)
    #fig, ax = plt.subplots()
    #plt.title(title)# + '\n' + subtitle)
    ##.apply(lambda x: x * 100)
    #plt.plot(x_data[1:100], np.array(y_data[1:100]) * 100, color=hpi_red, label="estimated counts")
    #plt.ylabel('Probability [%]')
    #plt.xlabel('Overcount')
    #ax.set_ylim(ymin=0)
    #ax.set_xlim(xmin=1)
    #ax.yaxis.grid()
    #ax.xaxis.grid()
    
    # Save Output
    #file_name = (title + '_' + subtitle).replace(" ", "_").replace(":", "_").replace(".","_") \
    #                                    .replace(",","_").replace("\n","_").replace("__", "_")
    #folder = "plots/cardinality_estimation/"
    #create_folder(folder)
    #plt.savefig(folder + '{}.pgf'.format(file_name))
    #plt.savefig(folder + '{}.pdf'.format(file_name))
    
    #Render
    #plt.show()

def evaluate(quotient_size, remainder_size, data_distribution, distinct_values):
    data_name = data_distribution + str(distinct_values)
    data = get_results("filter_cardinality_estimation_" + data_name)
    row_count = data["row_count"].iloc[0]
    selection = select(data, row_count, distinct_values, data_name, quotient_size, remainder_size)
    if len(selection) == 0:
        return
    
    title = "Filter Counts Vs. Actual Counts"
    subtitle = "Value Count: " + str(row_count) \
                + ", Distinct Values: " + str(distinct_values) + ", Data: " + data_distribution + '\n' \
                + "Quotient Size: " + str(quotient_size) + ", Remainder Size: " + str(remainder_size)
            
    fig, ax = plt.subplots()
    plt.title(title)# + '\n' + subtitle)
    plt.plot(selection["value"], selection["filter_count"], color=hpi_red, label="Filter Counts")
    plt.plot(selection["value"], selection["actual_count"], color=hpi_blue, label="Actual Counts")
    plt.legend(loc='upper left')
    plt.ylabel('Value Count')
    plt.xlabel('Distinct Values')
    ax.set_ylim(ymin=0)
    ax.set_xlim(xmin=0)
    ax.set_xlim(xmax=distinct_values)
    
    file_name = (title + '_' + subtitle).replace(" ", "_").replace(":", "_").replace(".","_") \
                    .replace(",","_").replace("\n","_").replace("__", "_")
    create_folder("plots/cardinality_estimation/")
    plt.savefig('plots/cardinality_estimation/{}.pgf'.format(file_name))
    plt.savefig('plots/cardinality_estimation/{}.pdf'.format(file_name))
    plt.show()
    
def filter_comparison_table(quotient_sizes, remainder_sizes, data_name, _count, distinct_values):
    data = get_results("misestimation")
    filter_information = []
    for quotient_size in quotient_sizes:
        for remainder_size in remainder_sizes:
            estimation_tec = "filter_" + str(quotient_size) + "_" + str(remainder_size)
            selection = select(data, row_count, distinct_values, data_name, estimation_tec)
            if len(selection) == 0:
                continue
            sample_size = selection["sample_size"].iloc[0]
            x_data = selection["error"]
            y_data = selection["occurrences"].apply(lambda x: x / sample_size)
            filter_size_bits = pow(2, quotient_size) * (2 + remainder_size)
            filter_size_bytes = filter_size_bits / 8
            bits_per_value = filter_size_bits / row_count
            mean_error = get_mean_error(x_data, y_data)
            mean_squared_error = get_mean_squared_error(x_data, y_data)
            correct_probability = y_data.iloc[0]
            filter_information.append((quotient_size, remainder_size, bits_per_value, mean_error, mean_squared_error))
    

    columns = ["Quotient Size", "Remainder Size", "Bits per Value", "Mean Error", "Mean Squared Error"]
    df = pd.DataFrame(filter_information, columns=columns)
    #display(df.sort_values(by=["Bits per Value"]))
    display(df)
    file_name = data_name + str(distinct_values) + "_filter_table"
    data_frame_to_tex(df, 'plots/cardinality_estimation/{}.tex'.format(file_name))
    

In [15]:
remainder_sizes = [2, 4, 8, 16]
quotient_sizes = [12, 13, 14, 15, 16, 17]
filter_comparison_table(quotient_sizes, remainder_sizes, "normal", 100000, 3000)

,Quotient Size,Remainder Size,Bits per Value,Mean Error,Mean Squared Error
0,14,4,0.98304,0.352340,19.986813
1,14,8,1.63840,0.023957,1.408457
2,14,16,2.94912,0.000360,0.016773
3,15,2,1.31072,0.749813,42.567713
4,15,4,1.96608,0.204323,11.650183
5,15,8,3.27680,0.012187,0.696927
6,15,16,5.89824,0.000000,0.000000
7,16,2,2.62144,0.386807,22.038973
8,16,4,3.93216,0.086947,4.969967
9,16,8,6.55360,0.006397,0.391877


In [23]:
remainder_sizes = [2, 4, 8, 16]
quotient_sizes = [12, 13, 14, 15, 16, 17]
filter_comparison_table(quotient_sizes, remainder_sizes, "uniform", 100000, 3000)

,Quotient Size,Remainder Size,Bits per Value,Mean Error,Mean Squared Error
0,14,4,0.98304,0.37972,12.68322
1,14,8,1.63840,0.02200,0.72600
2,14,16,2.94912,0.00000,0.00000
3,15,2,1.31072,0.75922,25.68588
4,15,4,1.96608,0.18172,5.99676
5,15,8,3.27680,0.00858,0.28314
6,15,16,5.89824,0.00000,0.00000
7,16,2,2.62144,0.36718,12.18228
8,16,4,3.93216,0.08954,2.97660
9,16,8,6.55360,0.00594,0.19602


In [3]:
remainder_sizes = [2, 4, 8, 16]
quotient_sizes = [12, 13, 14]
data_name = "zipf"
row_count = 100000
distinct_values = 3000

for quotient_size in quotient_sizes:
    for remainder_size in remainder_sizes:
        estimation_tec = "filter_" + str(quotient_size) + "_" + str(remainder_size)
        #evaluate(quotient_size, remainder_size, data_name, distinct_values)
        misestimation_evaluation(row_count, distinct_values, data_name, estimation_tec)
        
misestimation_evaluation(row_count, distinct_values, data_name, "postgres1_10")
misestimation_evaluation(row_count, distinct_values, data_name, "postgres2_10")

AttributeError: 'DataFrame' object has no attribute 'row_count'

In [54]:
remainder_sizes = [2, 4, 8, 16]
quotient_sizes = [14, 15, 16, 17]
data_name = "uniform"
distinct_count = 3000
row_count = 100000

for quotient_size in quotient_sizes:
    for remainder_size in remainder_sizes:
        estimation_tec = "filter_" + str(quotient_size) + "_" + str(remainder_size)
        misestimation_evaluation(row_count, distinct_values, data_name, estimation_tec)

Estimation Technique: filter_14_4
Sample Size: 300000
Mean Error: 0.37972
Mean Squared Error: 12.68322
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_14_8
Sample Size: 300000
Mean Error: 0.022
Mean Squared Error: 0.726
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_14_16
Sample Size: 300000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_15_2
Sample Size: 300000
Mean Error: 0.75922
Mean Squared Error: 25.68588
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_15_4
Sample Size: 300000
Mean Error: 0.18172
Mean Squared Error: 5.99676
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_15_8
Sample Size: 300000
Mean Error: 0.00858
Mean Squared Error: 0.28314
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_15_16
Sample Size: 300000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_16_2
Sample Size: 300000
Mea

In [56]:
data = "normal"
distinct_values = 3000
row_count = 100000

#evaluate(14, 4, data, distinct_values)
misestimation_evaluation(row_count, distinct_values, data_name, "filter_14_4")
#evaluate(14, 8, data, distinct_values)
misestimation_evaluation(row_count, distinct_values, data_name, "filter_14_8")
#evaluate(14, 16, data, distinct_values)
misestimation_evaluation(row_count, distinct_values, data_name, "filter_14_16")
misestimation_evaluation(row_count, distinct_values, data_name, "postgres1_10")
misestimation_evaluation(row_count, distinct_values, data_name, "postgres2_10")

Estimation Technique: filter_14_4
Sample Size: 300000
Mean Error: 0.37972
Mean Squared Error: 12.68322
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_14_8
Sample Size: 300000
Mean Error: 0.022
Mean Squared Error: 0.726
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_14_16
Sample Size: 300000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0

Estimation Technique: postgres1_10
Sample Size: 3000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0

Estimation Technique: postgres2_10
Sample Size: 3000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0



In [57]:
data = "uniform"
distinct_values = 3000

#evaluate(14, 4, data, distinct_values)
misestimation_evaluation(row_count, distinct_values, data_name, "filter_14_4")
#evaluate(14, 8, data, distinct_values)
misestimation_evaluation(row_count, distinct_values, data_name, "filter_14_8")
#evaluate(14, 16, data, distinct_values)
misestimation_evaluation(row_count, distinct_values, data_name, "filter_14_16")
misestimation_evaluation(row_count, distinct_values, data_name, "postgres1_10")
misestimation_evaluation(row_count, distinct_values, data_name, "postgres2_10")

Estimation Technique: filter_14_4
Sample Size: 300000
Mean Error: 0.37972
Mean Squared Error: 12.68322
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_14_8
Sample Size: 300000
Mean Error: 0.022
Mean Squared Error: 0.726
Column Size [kB](uint16_t): 200.0

Estimation Technique: filter_14_16
Sample Size: 300000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0

Estimation Technique: postgres1_10
Sample Size: 3000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0

Estimation Technique: postgres2_10
Sample Size: 3000
Mean Error: 0.0
Mean Squared Error: 0.0
Column Size [kB](uint16_t): 200.0

